<a href="https://colab.research.google.com/github/jittapont/bank_proof_classification/blob/master/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

## Import library

Note need to use tensorflow version 1.x

In [1]:
%tensorflow_version 1.x
import pickle
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical
import datetime

TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
tf.__version__

'1.15.2'

## Load datasets

In [5]:
with open("/content/drive/My Drive/Dataset/X.pickle","rb") as pickle_in:
    X = pickle.load(pickle_in)

In [7]:
with open("/content/drive/My Drive/Dataset/y.pickle","rb") as pickle_in:
    y = pickle.load(pickle_in)

In [8]:
X.shape

(220, 100, 100, 1)

In [9]:
y = to_categorical(y)

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn't guaranteed
gpu = GPUs[0]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=5b23d258aca0774453554bfd0f047ff8f3ee71e02b6bc72ddbf117c628f591cf
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 22.6 GB  I Proc size: 7.4 GB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


## Train model

In [11]:
model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64))
model.add(Dense(64))

model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=128, epochs=30, validation_split=0.3)

model_path = r'/content/drive/My Drive/Dataset'
model.save(os.path.join(
    model_path,
    f"month_classification_model_{datetime.datetime.now().strftime('%Y-%m-%d')}.model")
)

Train on 154 samples, validate on 66 samples
Epoch 1/30
154/154 [==============================] - 1s 3ms/sample - loss: 2.6883 - acc: 0.1299 - val_loss: 2.3746 - val_acc: 0.2576
Epoch 2/30
154/154 [==============================] - 0s 2ms/sample - loss: 2.4680 - acc: 0.1948 - val_loss: 2.4773 - val_acc: 0.0606
Epoch 3/30
154/154 [==============================] - 0s 2ms/sample - loss: 2.3920 - acc: 0.1039 - val_loss: 2.2569 - val_acc: 0.2576
Epoch 4/30
154/154 [==============================] - 0s 2ms/sample - loss: 2.1746 - acc: 0.3442 - val_loss: 1.9726 - val_acc: 0.1212
Epoch 5/30
154/154 [==============================] - 0s 2ms/sample - loss: 1.8801 - acc: 0.2857 - val_loss: 1.1544 - val_acc: 0.7727
Epoch 6/30
154/154 [==============================] - 0s 2ms/sample - loss: 1.0272 - acc: 0.8247 - val_loss: 0.8294 - val_acc: 0.7273
Epoch 7/30
154/154 [==============================] - 0s 2ms/sample - loss: 0.5854 - acc: 0.8247 - val_loss: 0.2486 - val_acc: 0.9242
Epoch 8/30
154/15

## Save model

In [14]:
model_path = '/content/drive/My Drive/Dataset'
model_name = f"image_classification_model_{datetime.datetime.now().strftime('%Y-%m-%d')}.model"
model.save(os.path.join(model_path, model_name))

## Test model prediction

In [17]:
import cv2
import tensorflow as tf
import numpy as np
categories = [
        "BAAC",
        "BAY",
        "BBL",
        "GSB",
        "KBANK",
        "KTB",
        "PAOTANG",
        "Others",
        "SCB",
        "TBANK",
        "TMB"
]

def prepare(filepath):
    IMG_SIZE = 100
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) 
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1) 

model = tf.keras.models.load_model(os.path.join(model_path, model_name))
prediction = model.predict([prepare('/content/drive/My Drive/Dataset/1.jpeg')])
print(prediction[0])
print(categories[np.argmax(prediction[0])])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
KBANK
